# Creating a CNN

### Import Libraries

In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras import Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

### Prepare the Dataset

We will be making use of the *Cifar-10* dataset, which consists of 32x32 pixel images.

Divide the dataset into training and testing sets. Normalise the data into the range (0, 1) by dividing with the highest pixel value (0), then convert to vectors.

In [2]:
(trainimage, trainlabel), (testimage, testlabel) = cifar10.load_data()
trainimage, testimage=trainimage/255.0, testimage/255.0
trainlabelarray=to_categorical(trainlabel, 10)
testlabelarray=to_categorical(testlabel, 10)
classnames=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


### Create a Model

Create a model by stacking Conv2D layers.



In [4]:
def createModel():
  input=Input(shape=(32, 32, 3), name='input')
  conv1=Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), name='conv1')(input)
  pool1=MaxPooling2D(2, 2)(conv1)
  drop1=Dropout(0.5)(pool1)
  conv2=Conv2D(64, (3, 3), activation='relu', name='conv2')(drop1)
  pool2=MaxPooling2D(2, 2)(conv2)
  drop2=Dropout(0.5)(pool2)
  conv3=Conv2D(128, (3, 3), activation='relu', name='conv3')(drop2)
  flat=Flatten()(conv3)
  dense1=Dense(128, activation='relu', name='dense1')(flat)
  output=Dense(10, activation='softmax', name='output')(dense1)
  model=Model(inputs=input, outputs=output)
  return model

In [6]:
model=createModel()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
earlystopping=EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
history=model.fit(trainimage, trainlabelarray, epochs=100, shuffle=True, validation_split=0.2, callbacks=[earlystopping])

Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 79s 61ms/step - accuracy: 0.2966 - loss: 1.8860 - val_accuracy: 0.4934 - val_loss: 1.4148
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 78s 58ms/step - accuracy: 0.4998 - loss: 1.3757 - val_accuracy: 0.5457 - val_loss: 1.2807
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 82s 58ms/step - accuracy: 0.5578 - loss: 1.2286 - val_accuracy: 0.6196 - val_loss: 1.0804
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 82s 58ms/step - accuracy: 0.5967 - loss: 1.1343 - val_accuracy: 0.6190 - val_loss: 1.0966
Epoch 5/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 78s 63ms/step - accuracy: 0.6183 - loss: 1.0814 - val_accuracy: 0.6386 - val_loss: 1.0226
Epoch 6/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 96s 74ms/step - accuracy: 0.6311 - loss: 1.0405 - val_accuracy: 0.6695 - val_loss: 0.9493
Epoch 7/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 124s 60ms/step - accuracy: 0.6477 - loss: 0.9967 - val_accuracy: 0.6592 - val_loss: 0.9684
Epoch 8/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 84s 61ms/step - accuracy:

In [8]:
def testModel(model, x, y):
  ypred=model.predict(x)
  ypred=np.argmax(ypred, axis=1)
  print("confusion matrix\n", confusion_matrix(y, ypred))
  print("classification report\n", classification_report(y, ypred))


In [9]:
testModel(model, testimage, testlabel)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step
confusion matrix
 [[736  24  51  14  27   2  13   3 101  29]
 [ 11 871   4   8   8   5  12   1  32  48]
 [ 71   7 594  58 127  43  63   6  20  11]
 [ 22  12  98 517 122 125  54  10  21  19]
 [ 21   4  87  44 735  18  51  26  10   4]
 [ 14   3  70 208  79 546  43  17  11   9]
 [  4   8  48  53  41  14 815   3  11   3]
 [ 12   3  43  63 134  65  10 638   8  24]
 [ 54  29  11   9  12   5   4   2 862  12]
 [ 30 111  18  12  13   5  17   6  51 737]]
classification report
               precision    recall  f1-score   support

           0       0.75      0.74      0.75      1000
           1       0.81      0.87      0.84      1000
           2       0.58      0.59      0.59      1000
           3       0.52      0.52      0.52      1000
           4       0.57      0.73      0.64      1000
           5       0.66      0.55      0.60      1000
           6       0.75      0.81      0.78      1000
           7       0.90      0.64      0.75      100